<a href="https://colab.research.google.com/github/Ximena5745/Developer/blob/main/Anal%C3%ADsis%20de%20textos%20v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install xlsxwriter unidecode textblob plotly nltk
import nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 11.7 MB/s eta 0:00:00


In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [53]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter, defaultdict
import re
from unidecode import unidecode
from typing import List, Tuple, Dict
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
# Descargar las stopwords de nltk si no están disponibles
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
class CalidadAnalyzerPDI:
    def __init__(self, archivo_excel: str):
        """Inicializa el analizador"""
        self.archivo_excel = archivo_excel
        self.min_longitud_frase = 4
        self.similitud_umbral = 0.75
        self.df_calidad = self.cargar_datos()
        self.stopwords_es = stopwords.words("spanish")  # Lista de stopwords en español
        self.modelo_embeddings = SentenceTransformer("paraphrase-MiniLM-L6-v2")  # Modelo semántico
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.columnas_relevantes = [
            "Calidad", "Calidad2", "Expansión", "Expansión2",
            "Educación para toda la vida", "Educación para toda la vida2",
            "Experiencia", "Experiencia2",
            "Transformación Organizacional", "Transformación Organizacional2",
            "Sostenibilidad", "Sostenibilidad2"
        ]

    def cargar_datos(self):
        """Carga los datos desde un archivo Excel"""
        try:
            df = pd.read_excel(self.archivo_excel, engine='openpyxl')
            print(f"✓ Archivo cargado exitosamente con {len(df)} filas y {len(df.columns)} columnas")
            return df
        except Exception as e:
            print(f"✘ Error cargando datos: {str(e)}")
            raise

    def _procesar_texto(self, texto: str) -> str:
        """Limpia y normaliza un texto"""
        texto = unidecode(str(texto).lower())
        texto = re.sub(r"[^a-z\s]", " ", texto)
        return " ".join(texto.split())

    def _extraer_frases_validas(self, texto: str) -> List[str]:
        """Extrae frases relevantes de un texto"""
        frases = re.split(r'[.,;]', texto)
        return [
            frase.strip() for frase in frases
            if len(frase.split()) >= self.min_longitud_frase
        ]

    def _agrupar_frases_por_contexto(self, frases: List[str]) -> List[str]:
        """Agrupa frases similares semánticamente"""
        embeddings = self.modelo_embeddings.encode(frases, convert_to_tensor=True)
        similitudes = cosine_similarity(embeddings)

        grupos = defaultdict(list)
        procesadas = set()

        for i, frase in enumerate(frases):
            if i in procesadas:
                continue
            grupo_actual = [i]
            for j in range(i + 1, len(frases)):
                if j not in procesadas and similitudes[i, j] > self.similitud_umbral:
                    grupo_actual.append(j)
                    procesadas.add(j)

            # Seleccionar frase representativa del grupo
            frase_representativa = max(grupo_actual, key=lambda idx: len(frases[idx]))
            grupos[frase_representativa].extend(grupo_actual)

        # Construir resultado con las mejores frases
        frases_representativas = [frases[key] for key in grupos.keys()]
        return frases_representativas, grupos

    def _contar_ocurrencias(self, textos: List[str], grupos: dict, frases: List[str]) -> List[Tuple[str, int]]:
        """Cuenta todas las ocurrencias explícitas e implícitas de los grupos en los comentarios"""
        conteos = defaultdict(int)
        embeddings_grupos = self.modelo_embeddings.encode(frases, convert_to_tensor=True)

        for texto in textos:
            texto_limpio = self._procesar_texto(texto)
            frases_comentario = self._extraer_frases_validas(texto_limpio)
            if not frases_comentario:
                continue

            embeddings_comentario = self.modelo_embeddings.encode(frases_comentario, convert_to_tensor=True)
            similitudes = cosine_similarity(embeddings_comentario, embeddings_grupos)

            for i, frase in enumerate(frases_comentario):
                # Identificar el grupo más similar
                grupo_idx = np.argmax(similitudes[i])
                if similitudes[i, grupo_idx] > self.similitud_umbral:
                    conteos[frases[grupo_idx]] += 1

        # Consolidar frecuencias finales
        resultado = [(frase, conteos[frase]) for frase in frases]
        resultado.sort(key=lambda x: x[1], reverse=True)
        return resultado

    def analizar_columna(self, columna: str) -> List[Tuple[str, int]]:
        """Analiza una columna de comentarios y genera las frases más relevantes"""
        if columna not in self.df_calidad.columns:
            print(f"⚠️ Columna {columna} no encontrada en el archivo.")
            return []

        print(f"Analizando columna: {columna}")
        textos = self.df_calidad[columna].dropna().astype(str).tolist()
        frases_totales = []
        for texto in textos:
            texto_limpio = self._procesar_texto(texto)
            frases_totales.extend(self._extraer_frases_validas(texto_limpio))

        if not frases_totales:
            print(f"⚠️ No se encontraron frases válidas en la columna {columna}.")
            return []

        frases_unicas = list(set(frases_totales))
        frases_representativas, grupos = self._agrupar_frases_por_contexto(frases_unicas)
        resultado = self._contar_ocurrencias(textos, grupos, frases_representativas)

        return resultado

    def generar_reporte_excel(self) -> str:
        """Genera un archivo Excel con los resultados del análisis"""
        resultados = {}
        for columna in self.columnas_relevantes:
            if columna in self.df_calidad.columns:
                resultados[columna] = self.analizar_columna(columna)
            else:
                print(f"⚠️ Columna {columna} no encontrada en el archivo.")

        nombre_archivo = f"analisis_calidad_{self.timestamp}.xlsx"
        with pd.ExcelWriter(nombre_archivo, engine="xlsxwriter") as writer:
            for columna, frases in resultados.items():
                if frases:
                    df_resultados = pd.DataFrame(frases, columns=["Frase", "Frecuencia"])
                    df_resultados.to_excel(writer, sheet_name=columna[:31], index=False)

        print(f"✓ Análisis completado: {nombre_archivo}")
        return nombre_archivo


In [61]:
def ejecutar_analisis(archivo_excel: str):
    """Función principal para ejecutar el análisis"""
    try:
        analyzer = CalidadAnalyzerPDI(archivo_excel)
        reporte = analyzer.generar_reporte_excel()
        print(f"✓ Reporte generado: {reporte}")
    except Exception as e:
        print(f"✘ Error durante el análisis: {str(e)}")

if __name__ == "__main__":
    archivo = "Preguntas abiertas 19_11_2024.xlsx"  # Cambiar por el nombre correcto del archivo
    ejecutar_analisis(archivo)

✓ Archivo cargado exitosamente con 3777 filas y 18 columnas
Analizando columna: Calidad
Analizando columna: Calidad2
Analizando columna: Expansión
Analizando columna: Expansión2
Analizando columna: Educación para toda la vida
Analizando columna: Educación para toda la vida2
Analizando columna: Experiencia
Analizando columna: Experiencia2
Analizando columna: Transformación Organizacional
Analizando columna: Transformación Organizacional2
Analizando columna: Sostenibilidad
Analizando columna: Sostenibilidad2
✓ Análisis completado: analisis_calidad_20241124_070946.xlsx
✓ Reporte generado: analisis_calidad_20241124_070946.xlsx
